# Common workflow

In [ ]:
# This cell is not visible in the documentation
import logging

logging.getLogger().setLevel(logging.ERROR)

To install the `expertsystem` in Google Colab, **uncomment** the following:

In [ ]:
# !pip install expertsystem[doc]

Your analysis starts by defining an amplitude model that describes the
reaction process you want to study. Such a model is generally very complex
and requires a fair amount of effort by the analyst (you). This also gives a
lot of room for mistakes.

The [‘expert system’](https://en.wikipedia.org/wiki/Expert_system) is
responsible to give you advice on the form of an amplitude model based
on the problem set you define (initial state, final state, allowed
interactions, intermediate states, etc.). Internally, the system propagates
the quantum numbers through the reaction graph while satisfying the specified
conservation rules. How to control this procedure is explained in more detail
below.

Afterwards, the amplitude model of the expert system can be exported into [ComPWA](https://compwa.github.io/) or [Tensorwaves](https://pwa.readthedocs.io/projects/tensorwaves). The
model can for instance be used to generate a data set (toy Monte Carlo) for
this reaction and to optimize its parameters to resemble an actual data set
as good as possible.

## 1.1. Define the problem set

In [ ]:
from expertsystem.reaction import InteractionTypes, StateTransitionManager

stm = StateTransitionManager(
    initial_state=["J/psi(1S)"],
    final_state=["gamma", "pi0", "pi0"],
    allowed_intermediate_particles=[
        "f(0)(980)",
        "f(0)(1500)",
        "f(2)(1270)",
        "f(2)(1950)",
        "omega(782)",
        "D*(2007)0",
        "rho(770)0",
        "phi(1020)",
        "a(0)(980)0",
    ],
    formalism_type="helicity",
)

## 1.2. Prepare topologies

Create all topology graphs using the **isobar model** (tree of two-body
decays) and initialize the graphs with the initial and final state. Remember
that each interaction node defines its own set of conservation laws.

| Interaction          | Strength  |
| -------------------- | --------- |
| strong               | $60$      |
| electromagnetic (EM) | $1$       |
| weak                 | $10^{-4}$ |

In [ ]:
graph_interaction_settings_groups = stm.prepare_graphs()

## 1.3. Find solutions

In [ ]:
result = stm.find_solutions(graph_interaction_settings_groups)

In [ ]:
print("found", len(result.solutions), "solutions!")
result.get_intermediate_particles().names

Now we have a lot of solutions that are actually heavily suppressed
(they involve two weak decays).

In [ ]:
logging.getLogger().setLevel(logging.WARNING)

In [ ]:
stm.set_allowed_interaction_types([InteractionTypes.Strong])
graph_interaction_settings_groups = stm.prepare_graphs()
result = stm.find_solutions(graph_interaction_settings_groups)

print("found", len(result.solutions), "solutions!")
result.get_intermediate_particles().names

In [ ]:
logging.getLogger().setLevel(logging.CRITICAL)

Huh, what happened here? Actually, since a $\gamma$ particle appears in one of the interaction nodes, the expert system knows that this node **must involve EM interactions**! Because the node can be an effective interaction, the weak interaction cannot be excluded, as it contains only a subset of conservation laws.

Since only the strong interaction was supposed to be used, this results in a warning and the STM automatically corrects the mistake.

Once the EM interaction is included, this warning disappears. Be aware, however, that the EM interaction is now available globally. Hence, there now might be solutions in which both nodes are electromagnetic.

In [ ]:
stm.set_allowed_interaction_types([InteractionTypes.Strong, InteractionTypes.EM])
graph_interaction_settings_groups = stm.prepare_graphs()
result = stm.find_solutions(graph_interaction_settings_groups)

print("found", len(result.solutions), "solutions!")
result.get_intermediate_particles().names

Great! Now we selected only the strongest contributions. Be aware, though, that there are more effects that can suppress certain decays, like small branching ratios. In this example, the initial state $J/\Psi$ can decay into $\pi^0 + \rho^0$ or $\pi^0 + \omega$.

| decay                               | branching ratio |
| ----------------------------------- | --------------- |
| $$\omega \rightarrow \gamma+\pi^0$$ | 0.0828          |
| $$\rho^0 \rightarrow \gamma+\pi^0$$ | 0.0006          |

Unfortunately, the $\rho^0$ mainly decays into $\pi+\pi$, not $\gamma+\pi^0$ and is therefore suppressed. This information is currently not known to the expert system, but it is possible to hand the expert system a list of allowed intermediate states.

In [ ]:
# particles are found by name comparison,
# i.e. f2 will find all f2's and f all f's independent of their spin
stm.allowed_intermediate_particles = ["f(0)", "f(2)"]

result = stm.find_solutions(graph_interaction_settings_groups)

print("found", len(result.solutions), "solutions!")
result.get_intermediate_particles().names

Now we have selected all amplitudes that involve **f** states. The warnings appear only to notify the user that the list of solutions is not exhaustive: for certain edges in the graph, no suitable particle was found (since only f states were allowed).

## 1.4. Generate an amplitude model

In [ ]:
import expertsystem as es

amplitude_model = es.amplitude.generate(result)
list(amplitude_model.parameters)

In [ ]:
es.io.write(amplitude_model, filename="model.xml")
es.io.write(amplitude_model, filename="model.yml")

In [ ]:
imported_model_xml = es.io.load_amplitude_model(filename="model.xml")
imported_model_yml = es.io.load_amplitude_model(filename="model.yml")

assert imported_model_xml == amplitude_model
assert imported_model_yml == amplitude_model

Have a look through the sections of the resulting XML or YAML recipe file to see what you recognize from the problem set defined above. There may also be some things you want to change in there manually, so **make sure you store this recipe file** carefully (e.g. track it with Git) as to avoid overwriting it your changes after rerunning the expert system.

Now you can use this recipe file as an amplitude model in a PWA framework!